In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib import image
from matplotlib import pyplot
import json
import os
import cv2
import random
import concurrent.futures
import time
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

print(tf.__version__)

In [ ]:
train = pd.read_csv("../input/hotel-id-csv-sorted-appended/x.csv", dtype=str)

In [ ]:
classes=np.unique(train['hotel_id'].values.astype(int))
dic = dict()
i = 0
for cs in classes:
    dic.update({str(cs): i})
    i+=1

In [ ]:
labels=[]
for ele in train['hotel_id'].values:
    labels.append(dic[ele])

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(labels),
                                                 labels)
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
# Splitting into train and validation sets
names = train['name_path'].values
VAL_SPLIT = 0.25
train_names, val_names, train_labels, val_labels = train_test_split(names[:10000], labels[:10000], \
                                                    test_size=VAL_SPLIT, random_state=42,\
                                                                       stratify=labels[:10000])

train_labels = np.array(train_labels)
val_labels = np.array(val_labels)

In [ ]:
IMSIZE = 256
def read_img(image):
    img = tf.keras.preprocessing.image.load_img(image, color_mode='rgb', target_size=(IMSIZE, IMSIZE))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.image.convert_image_dtype(img, dtype=tf.uint8, saturate=True)
    return img
def prepare_dataset(namelist, labels, path):
    start = time.time()
    labels = np.array(labels)
    labels = tf.convert_to_tensor(labels)
    labels = tf.cast(labels, tf.uint16)
    imgs = []
    with concurrent.futures.ThreadPoolExecutor(max_workers = 16) as executor:
        i = 0
        for value in executor.map(read_img, namelist):
            i+=1
            print("\rFetching: [{}/{}]".format(i, len(namelist)), end="", flush=True)
            imgs.append(value)
        imgs = tf.convert_to_tensor(imgs)
    print("\nExecution time: ",time.time() - start, "s")
    return imgs, labels

In [ ]:
with tf.device('/cpu:0'):
    path = '../input/hotel-id-2021-fgvc8'
    print("Training count", len(train_names))
    print("Validation count", len(val_names))
    train_images, train_labels = prepare_dataset(train_names, train_labels, path)
    #val_images, val_labels = prepare_dataset(val_names, val_labels, path)

In [ ]:
print("Training Image tensor shape", train_images.shape)
print("Training Labels tensor shape", train_labels.shape)
#print("Testing Image tensor shape", val_images.shape)
#print("Tesing Labels tensor shape", val_labels.shape)

In [ ]:
"""Only use for TPU session"""
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
SEED = 1000
random_rotation = tf.keras.layers.experimental.preprocessing.RandomRotation(3.142/2, seed=SEED)
random_flip = tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical", seed=SEED)
random_zoom = tf.keras.layers.experimental.preprocessing.RandomZoom((0, 0.35), seed=SEED)
random_translate = tf.keras.layers.experimental.preprocessing.RandomTranslation((-0.25, 0.25), (-0.25, 0.25), seed=SEED)

def preprocess(imgs, label):
    imgs = random_rotation.call(imgs)
    imgs = random_flip.call(imgs)
    imgs = random_zoom.call(imgs)
    imgs = random_translate.call(imgs)
    return imgs, label

def normalize(imgs, label):
    return tf.cast(imgs, tf.float16)/255, label

strategy = tf.distribute.experimental.TPUStrategy(resolver)
with tf.device('/cpu:0'):
    TRAIN_BATCH_SIZE = 128
    VAL_BATCH_SIZE = 64
    TRAIN_SIZE = len(train_images)
    VAL_SIZE = len(val_names)
    train_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(train_images, tf.uint8), \
                                                        tf.cast(train_labels, tf.uint16)))
    del train_images, train_labels
    train_dataset = train_dataset.shuffle(TRAIN_SIZE).repeat()\
                                 .batch(TRAIN_BATCH_SIZE)\
                                 .shuffle(TRAIN_BATCH_SIZE)
    #                .map(preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)\
    #                .cache()
    train_dataset = train_dataset.map(normalize, \
                                      num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                                 .prefetch(tf.data.AUTOTUNE)

In [ ]:
with tf.device('/cpu:0'):
    val_images, val_labels = prepare_dataset(val_names, val_labels, path)

In [ ]:
print("Testing Image tensor shape", val_images.shape)
print("Tesing Labels tensor shape", val_labels.shape)

In [ ]:
with tf.device('/cpu:0'):
    val_dataset = tf.data.Dataset.from_tensor_slices((tf.cast(val_images, tf.uint8), \
                                                      tf.cast(val_labels, tf.uint16)))
    del val_images, val_labels
    val_dataset = val_dataset.repeat().batch(VAL_BATCH_SIZE)
    val_dataset = val_dataset.map(normalize, \
                                  num_parallel_calls=tf.data.experimental.AUTOTUNE)\
                             .prefetch(tf.data.AUTOTUNE)

In [ ]:
def IResNet_brain_module(input, layercount):
    n = layercount
    ############################################################################
    # Parallel Block 1
    x1_1 = tf.keras.layers.Conv2D(n, (3, 3), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(input)
    x1_2 = tf.keras.layers.SeparableConv2D(n, (1, 1), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(x1_1)
    x1 = tf.keras.layers.add([x1_1, x1_2])
    ############################################################################
    # Parallel Block 2
    x2_1 = tf.keras.layers.Conv2D(n, (5, 5), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(input)
    x2_2 = tf.keras.layers.SeparableConv2D(n, (1, 1), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(x2_1)
    x2 = tf.keras.layers.add([x2_1, x2_2, x1])
    ############################################################################
    # Parallel Block 3
    x3_1 = tf.keras.layers.Conv2D(n, (1, 1), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(input)
    x3_2 = tf.keras.layers.SeparableConv2D(n, (1, 1), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(input)
    x3 = tf.keras.layers.add([x3_1, x3_2, x2, x1])
    ############################################################################
    # Parallel Block 4
    x4_1 = tf.keras.layers.Conv2D(n, (7, 7), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(input)
    x4_2 = tf.keras.layers.SeparableConv2D(n, (1, 1), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(x4_1)
    x4 = tf.keras.layers.add([x4_1, x4_2, x3, x2, x1])
    ############################################################################
    mod = tf.keras.layers.concatenate([x1, x2, x3, x4], axis = -1)
    mod = tf.keras.layers.BatchNormalization()(mod)
    return mod
def IResNet_connection_module(input1, input2, input3, input4, layercount):
    n = layercount
    ############################################################################
    # Parallel Block 1
    x1 = tf.keras.layers.Conv2D(n, (3, 3), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(input1)
    ############################################################################
    # Parallel Block 2
    x2 = tf.keras.layers.Conv2D(n, (3, 3), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(input2)
    ############################################################################
    # Parallel Block 3
    x3 = tf.keras.layers.Conv2D(n, (3, 3), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(input3)
    ############################################################################
    # Parallel Block 4
    x4 = tf.keras.layers.Conv2D(n, (3, 3), activation = 'relu', padding = 'same', kernel_regularizer='l2', bias_regularizer='l1')(input4)
    ############################################################################
    mod = tf.keras.layers.add([x1, x2, x3, x4])
    mod = tf.keras.layers.BatchNormalization()(mod)
    return mod
def IResNet_reduction_module(input, layercount):
    n = layercount
    ############################################################################
    # Reduction module
    R1= tf.keras.layers.SeparableConv2D(n, (5, 5), strides = 2, activation = 'relu', kernel_regularizer='l2', bias_regularizer='l1')(input)
    R1 = tf.keras.layers.BatchNormalization()(R1) 
    R1 = tf.keras.layers.SeparableConv2D(n, (3, 3), strides = 2, activation = 'relu', kernel_regularizer='l2', bias_regularizer='l1')(R1)
    mod = tf.keras.layers.BatchNormalization()(R1)
    R1 = tf.keras.layers.SeparableConv2D(2*n, (3, 3), strides = 2, activation = 'relu', kernel_regularizer='l2', bias_regularizer='l1')(R1)
    mod = tf.keras.layers.BatchNormalization()(R1)
    return mod
def IResNet_classifier_module(input, num_classes, activation):
    ############################################################################
    # Classifier module
    R1= tf.keras.layers.Flatten()(input)
    mod = tf.keras.layers.Dense(num_classes, activation=activation)(R1)
    return mod

In [ ]:
IMSIZE = 256
CHANNEL = 3
with strategy.scope():
    inp = tf.keras.layers.Input(shape=(IMSIZE, IMSIZE, CHANNEL))
    #img_inputs_1 = tf.keras.layers.experimental.preprocessing.RandomRotation(3.142/2, seed=SEED)(inp)
    #img_inputs_2 = tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical", seed=SEED)(img_inputs_1)
    #img_inputs_3 = tf.keras.layers.experimental.preprocessing.RandomZoom((0, 0.35), seed=SEED)(img_inputs_2)
    #connect = IResNet_connection_module(img_inputs_1, img_inputs_2, img_inputs_3, inp, 32)
    module = IResNet_brain_module(inp, 32) 
    red = IResNet_reduction_module(module, 64)
    red = IResNet_reduction_module(red, 64)
    cla = IResNet_classifier_module(red, 7770, 'softmax')
    model = tf.keras.models.Model(inputs=inp, outputs=cla, name = "IResNetv1")
    model.summary()
    tf.keras.utils.plot_model(model, show_shapes=True,to_file='./img.png')
    metric_auc = tf.keras.metrics.AUC(num_thresholds=200, name='auc')
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),\
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), \
              metrics=['acc'])

In [ ]:
IMSIZE = 256
CHANNEL = 3
import tensorflow_addons as tfa

with strategy.scope():
    base_model = tf.keras.applications.InceptionV3(include_top=False,\
                                                   weights='imagenet', pooling = 'max')
    base_model.trainable = True
    for layer in base_model.layers[:100]:
        layer.trainable = False
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(IMSIZE, IMSIZE, CHANNEL)),
        #tf.keras.layers.experimental.preprocessing.RandomRotation(3.142/2, seed=SEED),
        #tf.keras.layers.experimental.preprocessing.RandomFlip(mode="horizontal_and_vertical", seed=SEED),
        #tf.keras.layers.experimental.preprocessing.RandomZoom((0, 0.35), seed=SEED),
        #tf.keras.layers.experimental.preprocessing.RandomTranslation((-0.25, 0.25), (-0.25, 0.25), seed=SEED),
        base_model,
        tf.keras.layers.Dense(7770, activation='softmax')
    ])
    metric_auc = tf.keras.metrics.AUC(num_thresholds=200, name='auc')
    model.compile(optimizer=tf.keras.optimizers.RMSprop(),\
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), \
              metrics=['acc'])


In [ ]:
model_checkpoint_callback_acc = tf.keras.callbacks.ModelCheckpoint(
    filepath='./best-acc-model.h5',
    save_weights_only=False,
    monitor='val_acc',
    mode='max',
    save_best_only=True)
model_checkpoint_callback_loss = tf.keras.callbacks.ModelCheckpoint(
    filepath='./best-loss-model.h5',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)
model_checkpoint_callback_auc = tf.keras.callbacks.ModelCheckpoint(
    filepath='./best-auc-model.h5',
    save_weights_only=False,
    monitor='val_auc',
    mode='max',
    save_best_only=True)
lr_decay_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.85,
    patience=4,
    verbose=1,
    mode="min",
)
dist_train_dataset = strategy.experimental_distribute_dataset(train_dataset)
dist_val_dataset = strategy.experimental_distribute_dataset(val_dataset)

In [ ]:
history = model.fit(train_dataset, steps_per_epoch = int(TRAIN_SIZE/TRAIN_BATCH_SIZE), \
                    validation_data=val_dataset, validation_steps=int(VAL_SIZE/VAL_BATCH_SIZE),\
                    epochs=150, callbacks=[model_checkpoint_callback_acc, \
                                           model_checkpoint_callback_auc, \
                                           model_checkpoint_callback_loss, \
                                           lr_decay_plateau])